<a href="https://colab.research.google.com/github/Srishtimend/CS4400-Final/blob/main/solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
from google.colab import drive
drive.mount('/content/gdrive/')
%cd /content/gdrive/MyDrive/Colab\ Notebooks/Final Project
!pip install python-Levenshtein scikit-learn pandas numpy
# import pandas as pd
# import numpy as np
# from os.path import join
from keras.models import Sequential
from keras.layers import Dense



import pandas as pd
import numpy as np
from os.path import join

# 1. read data

ltable = pd.read_csv(join('data', "ltable.csv"))
rtable = pd.read_csv(join('data', "rtable.csv"))
train = pd.read_csv(join('data', "train.csv"))

from sklearn.model_selection import train_test_split



# 2. blocking
def pairs2LR(ltable, rtable, candset):
    ltable.index = ltable.id
    rtable.index = rtable.id
    pairs = np.array(candset)
    tpls_l = ltable.loc[pairs[:, 0], :]
    tpls_r = rtable.loc[pairs[:, 1], :]
    tpls_l.columns = [col + "_l" for col in tpls_l.columns]
    tpls_r.columns = [col + "_r" for col in tpls_r.columns]
    tpls_l.reset_index(inplace=True, drop=True)
    tpls_r.reset_index(inplace=True, drop=True)
    LR = pd.concat([tpls_l, tpls_r], axis=1)
    return LR


def block_by_brand(ltable, rtable):
    # ensure brand is str
    ltable['brand'] = ltable['brand'].astype(str)
    rtable['brand'] = rtable['brand'].astype(str)

    # get all brands
    brands_l = set(ltable["brand"].values)
    brands_r = set(rtable["brand"].values)
    brands = brands_l.union(brands_r)

    # map each brand to left ids and right ids
    brand2ids_l = {b.lower(): [] for b in brands}
    brand2ids_r = {b.lower(): [] for b in brands}
    for i, x in ltable.iterrows():
        brand2ids_l[x["brand"].lower()].append(x["id"])
    for i, x in rtable.iterrows():
        brand2ids_r[x["brand"].lower()].append(x["id"])

    # put id pairs that share the same brand in candidate set
    candset = []
    for brd in brands:
        l_ids = brand2ids_l[brd]
        r_ids = brand2ids_r[brd]
        for i in range(len(l_ids)):
            for j in range(len(r_ids)):
                candset.append([l_ids[i], r_ids[j]])
    return candset

# blocking to reduce the number of pairs to be compared
candset = block_by_brand(ltable, rtable)
print("number of pairs originally", ltable.shape[0] * rtable.shape[0])
print("number of pairs after blocking",len(candset))
candset_df = pairs2LR(ltable, rtable, candset)



# 3. Feature engineering
import Levenshtein as lev
from sklearn.metrics.pairwise import euclidean_distances


def jaccard_similarity(row, attr):
    x = set(row[attr + "_l"].lower().split())
    y = set(row[attr + "_r"].lower().split())
    return len(x.intersection(y)) / max(len(x), len(y))

import nltk
# Python3 implementation of above approach
from math import floor, ceil

# Function to calculate the
# Jaro Similarity of two s
def jaro_distance(s1, s2):
	
	# If the s are equal
	if (s1 == s2):
		return 1.0

	# Length of two s
	len1 = len(s1)
	len2 = len(s2)

	# Maximum distance upto which matching
	# is allowed
	max_dist = floor(max(len1, len2) / 2) - 1

	# Count of matches
	match = 0

	# Hash for matches
	hash_s1 = [0] * len(s1)
	hash_s2 = [0] * len(s2)

	# Traverse through the first
	for i in range(len1):

		# Check if there is any matches
		for j in range(max(0, i - max_dist),
					min(len2, i + max_dist + 1)):
			
			# If there is a match
			if (s1[i] == s2[j] and hash_s2[j] == 0):
				hash_s1[i] = 1
				hash_s2[j] = 1
				match += 1
				break

	# If there is no match
	if (match == 0):
		return 0.0

	# Number of transpositions
	t = 0
	point = 0

	# Count number of occurances
	# where two characters match but
	# there is a third matched character
	# in between the indices
	for i in range(len1):
		if (hash_s1[i]):

			# Find the next matched character
			# in second
			while (hash_s2[point] == 0):
				point += 1

			if (s1[i] != s2[point]):
				point += 1
				t += 1
	t = t//2

	# Return the Jaro Similarity
	return (match/ len1 + match / len2 +
			(match - t + 1) / match)/ 3.0

# Driver code
s1 = "CRATE"
s2 = "TRACE"

# Prjaro Similarity of two s
print(round(jaro_distance(s1, s2),6))

# This code is contributed by mohit kumar 29

def jaro_winkler(row, attr):
   x = row[attr + "_l"].lower()
   y = row[attr + "_r"].lower() 
   return jaro_distance(x,y)

def feature_engineering(LR):
    LR = LR.astype(str)
    attrs = ["title", "category", "brand", "modelno", "price"]
    features = []
    for attr in attrs:
        j_sim = LR.apply(jaccard_similarity, attr=attr, axis=1)
        l_dist = LR.apply(jaro_winkler, attr=attr, axis=1)
        features.append(j_sim)
        features.append(l_dist)
    features = np.array(features).T
    return features
candset_features = feature_engineering(candset_df)

# also perform feature engineering to the training set
training_pairs = list(map(tuple, train[["ltable_id", "rtable_id"]].values))
training_df = pairs2LR(ltable, rtable, training_pairs)
training_features = feature_engineering(training_df)
training_label = train.label.values

# 4. Model training and prediction
def modelTraining():
  model = Sequential()
  model.add(Dense(12, input_dim=10, activation="relu"))
  model.add(Dense(8, activation='relu'))
  model.add(Dense(1, activation='sigmoid')) #normalize output
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

def fitModel(training_features, training_label, candset_features, model):
  #training_features, training_label, y_train, y_test = train_test_split(training_features, training_label, test_size = 0.2, shuffle = True)
  model.fit(training_features, training_label, epochs=150, batch_size=10)
  y_pred = model.predict_classes(candset_features)

  return y_pred

#from sklearn.model_selection import train_test_split
#x_train,x_test, y_train, y_test = train_test_split(training_features, training_label, test_size = 0.2, random_state = 3)

model = modelTraining()
y_pred = fitModel(training_features,training_label, candset_features, model)
#y_pred = naiveTraining(training_features,training_label, candset_features)
#y_pred = naiveTraining(x_train,y_train, x_test, y_test)


# 5. output
matching_pairs = candset_df.loc[y_pred == 1, ["id_l", "id_r"]]
matching_pairs = list(map(tuple, matching_pairs.values))
matching_pairs_in_training = training_df.loc[training_label == 1, ["id_l", "id_r"]]
matching_pairs_in_training = set(list(map(tuple, matching_pairs_in_training.values)))
pred_pairs = [pair for pair in matching_pairs if
              pair not in matching_pairs_in_training]  # remove the matching pairs already in training
pred_pairs = np.array(pred_pairs)
pred_df = pd.DataFrame(pred_pairs, columns=["ltable_id", "rtable_id"])
pred_df.to_csv("output.csv", index=False)






Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
/content/gdrive/MyDrive/Colab Notebooks/Final Project
number of pairs originally 56376996
number of pairs after blocking 256606
0.733333
Epoch 1/150
500/500 [==============================] - 1s 1ms/step - loss: 0.3666 - accuracy: 0.8993
Epoch 2/150
500/500 [==============================] - 1s 1ms/step - loss: 0.2181 - accuracy: 0.9439
Epoch 3/150
500/500 [==============================] - 1s 1ms/step - loss: 0.2100 - accuracy: 0.9448
Epoch 4/150
500/500 [==============================] - 1s 1ms/step - loss: 0.1945 - accuracy: 0.9507
Epoch 5/150
500/500 [==============================] - 1s 1ms/step - loss: 0.2222 - accuracy: 0.9356
Epoch 6/150
500/500 [==============================] - 1s 1ms/step - loss: 0.1936 - accuracy: 0.9453
Epoch 7/150
500/500 [==============================] - 1s 1ms/step - loss: 0.2052 - accuracy: 0.9420
Epoch 8/150
500/500 [==

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [39]:
y_pred

array([[0.03912979],
       [0.03499398],
       [0.03887129],
       ...,
       [0.02819926],
       [0.0592491 ],
       [0.01696089]], dtype=float32)

In [52]:
#num_matches = 3000
#res = sorted(range(len(y_pred)), key = lambda sub: y_pred[sub], reverse=False)[-num_matches:]
